In [1]:
# Adapted from
# https://github.com/salvadorgarciamunoz/kipet/blob/master/kipet/examples/Ex_12_multexp_conc_diffreact.py
#

In [2]:
from __future__ import print_function
from kipet.library.TemplateBuilder import *
from kipet.library.PyomoSimulator import *
from kipet.library.ParameterEstimator import *
from kipet.library.VarianceEstimator import *
from kipet.library.data_tools import *
from kipet.library.MultipleExperimentsEstimator import *
import matplotlib.pyplot as plt
import os
import sys
import inspect
import six
import pandas as pd

In [3]:
builder = TemplateBuilder()  
    
builder.add_mixture_component('A',1.0)
builder.add_mixture_component('B',2.0)
builder.add_mixture_component('C',0.0)
builder.add_mixture_component('D',0.0)
    
builder.add_parameter('k1',init = 0.1, bounds = (0.0, 10.0))
builder.add_parameter('k2',init = 0.1, bounds = (0.0, 10.0))
    
def rule_odes(m,t):
    exprs = dict()
    exprs['A'] = -m.P['k1']*m.Z[t,'A']*m.Z[t,'B']
    exprs['B'] = -m.P['k1']*m.Z[t,'A']*m.Z[t,'B']-m.P['k2']*m.Z[t,'B']*m.Z[t,'C']
    exprs['C'] = m.P['k1']*m.Z[t,'A']*m.Z[t,'B']-m.P['k2']*m.Z[t,'B']*m.Z[t,'C']
    exprs['D'] = m.P['k2']*m.Z[t,'B']*m.Z[t,'C']
    return exprs

builder.set_odes_rule(rule_odes)

In [4]:
builder1 = TemplateBuilder()  
    
builder1.add_mixture_component('A',1.0)
builder1.add_mixture_component('B',1.0)
builder1.add_mixture_component('C',0.0)
builder1.add_mixture_component('D',0.0)
    
builder1.add_parameter('k1',init = 0.1, bounds = (0.0, 10.0))
builder1.add_parameter('k2',init = 0.1, bounds = (0.0, 10.0))

builder1.set_odes_rule(rule_odes)

In [7]:
C_frame1 = read_file('ABCD_cb0_2.csv', directory = '../my_data_sets')
C_frame2 = read_file('ABCD_cb0_1.csv', directory = '../my_data_sets')

read dir : /home/paperspace/learn_kipet/kipet_examples/../my_data_sets/ABCD_cb0_2.csv
read dir : /home/paperspace/learn_kipet/kipet_examples/../my_data_sets/ABCD_cb0_1.csv


In [9]:
start_time = {'Exp1':0.0, 'Exp2':0.0}
end_time = {'Exp1':4.0, 'Exp2':7.0}    
datasets = {'Exp1': C_frame1, 'Exp2': C_frame2}
sigmas = {'A':1, 'B':1, 'C':1, 'D':1}
variances = {'Exp1':sigmas, 'Exp2':sigmas}
builder_dict = {'Exp1':builder, 'Exp2':builder1}

In [12]:
options = dict()
options['linear_solver'] = 'ma27'

In [13]:
pest = MultipleExperimentsEstimator(datasets)
    
nfe = 50
ncp = 3

# Finally we run the parameter estimation. This solves each dataset separately first and then
# links the models and solves it simultaneously
results_pest = pest.run_parameter_estimation(solver = 'ipopt', 
                                                     tee=True,
                                                     nfe=nfe,
                                                     ncp=ncp,
                                                     covariance = False,
                                                     solver_opts = options,
                                                     start_time=start_time, 
                                                     end_time=end_time,
                                                     spectra_problem = False,
                                                     sigma_sq=variances,
                                                     builder = builder_dict)


SOLVING PARAMETER ESTIMATION FOR INDIVIDUAL DATASETS - For initialization

Solving for DATASET  Exp1
                 A         B         C         D
0.000000  1.003906  1.975654  0.039959  0.009157
0.311111  0.554819  1.383557  0.328321  0.194300
0.666667  0.375822  1.012860  0.272322  0.378839
0.977778  0.305279  0.864239  0.244618  0.496218
1.333333  0.269072  0.667304  0.151982  0.610147
1.644444  0.167628  0.489103  0.219111  0.629081
2.000000  0.150927  0.464735  0.170158  0.696073
2.311111  0.174195  0.365669  0.167863  0.778508
2.666667  0.111261  0.341744  0.113357  0.737370
2.977778  0.113912  0.310083  0.171967  0.777322
3.333333  0.088894  0.277996  0.147807  0.886791
3.644444  0.094967  0.276187  0.160119  0.845984
4.000000  0.025899  0.283118  0.028531  0.810664
Ipopt 3.12: linear_solver=ma27


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as

The estimated parameters are:
k1 1.127879512866723
k2 2.3717424295162086

SOLVING PARAMETER ESTIMATION FOR MULTIPLE DATASETS

Fixed parameters are:  []
Ipopt 3.12: linear_solver=ma27


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:    12118
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     1904

Total number of variables............................:     2414
                     variables with only lower bounds:        2
                variables with lower and uppe

In [14]:
print("The estimated parameters are:")

for k,v in results_pest.items():
    print(results_pest[k].P)

The estimated parameters are:
k1    1.022398
k2    1.960164
dtype: float64
k1    1.022398
k2    1.960164
dtype: float64


In [26]:
expt = 'Exp2'
fig, ax = plt.subplots()
for x in ['A', 'B', 'C', 'D']:
    ax.scatter(results_pest[expt].C.index, results_pest[expt].C[x])
    ax.plot(results_pest[expt].Z.index, results_pest[expt].Z[x])